# Multi-Agent LangGraph System for Research Paper Social Media Posts

This notebook implements a multi-agent LangGraph system that takes a research paper description and creates social media posts optimized for different platforms.

## System Architecture

The system consists of 4 main agents:
1. **SEARCH_AGENT**: Searches arXiv for research papers and extracts content
2. **SOCIAL_MEDIA_AGENT**: Creates social media posts from paper content
3. **COPY_EDITOR**: Refines posts to fit the target platform's tone
4. **SUPERVISOR**: Coordinates workflow between agents

## Input Parameters
- Research paper description
- Target social network (LinkedIn, X, Facebook)
- Objective of the social media post

## Output
- Social media post saved as markdown text file

## Setup and Dependencies

First, we'll set up our environment variables and import all necessary libraries.

## Installation

First, install the required Python packages:

In [11]:
!pip install langchain langchain-openai langchain-community langgraph langsmith arxiv

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6089 sha256=944dbd62340e514ade3e6bbee83a36aa0a1b3c8d0b5766722792421aedce4633
  Stored in directory: /Users/foohm/Library/Caches/pip/wheels/3d/4d/ef/37cdccc18d6fd7e0dd7817dcdf9146d4d6789c32a227a28134
Successfully built sgmllib3k

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import os
import getpass
from uuid import uuid4
from typing import TypedDict, Annotated, List
from pathlib import Path
import operator
import functools

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

## Environment Configuration

Set up LangSmith tracing for monitoring and debugging our multi-agent system.

**LangSmith Setup Instructions:**
1. Go to [LangSmith](https://smith.langchain.com/) and create an account
2. Create a new project or use an existing one
3. Navigate to Settings → API Keys
4. Create a new API key and copy it
5. Enter the API key when prompted below

LangSmith will automatically trace all agent interactions, tool usage, and workflow execution, providing valuable insights into your multi-agent system's performance.

In [13]:
# Set up LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE7 - Research Paper Social Media - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

In [14]:
# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

## Working Directory Setup

Create a directory for file operations where our agents will save and read files.

In [15]:
# Working directory for file operations
WORKING_DIRECTORY = Path("./output")
WORKING_DIRECTORY.mkdir(exist_ok=True)
print(f"Working directory: {WORKING_DIRECTORY.absolute()}")

Working directory: /Users/foohm/AIMakerSpace/AIE7/06_BonusAssignment/output


## State Definition

Define the state that will be passed between agents. This includes the conversation history, paper content, target platform, and workflow control.

In [16]:
class ResearchPaperSocialMediaState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    paper_text: str
    social_network: str
    objective: str
    social_media_post: str
    next: str

## File Management Tools

Create tools for reading, writing, and editing files. These will be used by our agents to manage paper content and social media posts.

In [17]:
@tool
def file_read(file_path: str) -> str:
    """Read a text file and return its contents as a string."""
    try:
        with open(WORKING_DIRECTORY / file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        return f"File {file_path} not found."
    except Exception as e:
        return f"Error reading file: {str(e)}"

@tool
def file_write(content: str, file_path: str) -> str:
    """Write content to a text file."""
    try:
        with open(WORKING_DIRECTORY / file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"Content successfully written to {file_path}"
    except Exception as e:
        return f"Error writing to file: {str(e)}"

@tool
def file_edit(file_path: str, line_number: int, new_content: str) -> str:
    """Insert content at a specific line in a text file."""
    try:
        with open(WORKING_DIRECTORY / file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        if line_number < 1 or line_number > len(lines) + 1:
            return f"Line number {line_number} is out of range."
        
        lines.insert(line_number - 1, new_content + '\n')
        
        with open(WORKING_DIRECTORY / file_path, 'w', encoding='utf-8') as f:
            f.writelines(lines)
        
        return f"Content inserted at line {line_number} in {file_path}"
    except Exception as e:
        return f"Error editing file: {str(e)}"

## Helper Functions

Create helper functions for agent creation and workflow management, following the patterns from the reference notebooks.

In [18]:
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str) -> AgentExecutor:
    """Create a function-calling agent with tools."""
    system_prompt += ("\nWork autonomously according to your specialty, using the tools available to you."
                     " Do not ask for clarification."
                     " Your other team members will collaborate with you with their own specialties."
                     " You are chosen for a reason!")
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def agent_node(state, agent, name):
    """Execute an agent and return the result as a message."""
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

def create_supervisor(llm: ChatOpenAI, system_prompt: str, members: List[str]) -> callable:
    """Create a supervisor agent that routes to team members."""
    options = ["FINISH"] + members
    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [{"enum": options}],
                },
            },
            "required": ["next"],
        },
    }
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}"),
    ]).partial(options=str(options), team_members=", ".join(members))
    
    return (
        prompt
        | llm.bind_functions(functions=[function_def], function_call="route")
        | JsonOutputFunctionsParser()
    )

## Initialize Language Model and Tools

Set up the language model and arXiv search tool that will be used across our agents.

In [19]:
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Initialize the arXiv search tool
arxiv_tool = ArxivQueryRun()

print("Language model and tools initialized successfully!")

Language model and tools initialized successfully!


## Agent Creation

Create our four specialized agents, each with specific tools and responsibilities.

### Search Agent

The Search Agent uses the arXiv tool to find research papers and save their content.

In [20]:
search_agent = create_agent(
    llm,
    [arxiv_tool, file_write],
    ("You are a research assistant specialized in finding academic papers on arXiv. "
     "When given a description of a paper, search for it using the arXiv tool and "
     "extract the full text content. Save the paper content to a file called 'paper_content.txt' for later use. "
     "Focus on finding the most relevant and recent papers. If you find multiple papers, "
     "choose the most relevant one based on the description provided.")
)

search_node = functools.partial(agent_node, agent=search_agent, name="SEARCH_AGENT")
print("Search Agent created successfully!")

Search Agent created successfully!


### Social Media Agent

The Social Media Agent reads research papers and creates engaging social media posts.

In [21]:
social_media_agent = create_agent(
    llm,
    [file_read, file_write],
    ("You are a social media content creator specialized in translating complex academic "
     "research into engaging social media posts. You understand how to adapt content for "
     "different platforms (LinkedIn, X/Twitter, Facebook) and create posts that match the "
     "target objective. Read the research paper content from 'paper_content.txt' and create "
     "compelling social media posts. Save your post to 'social_media_post.txt'. "
     "Consider the target social network and objective when crafting your post. "
     "Make it engaging, informative, and appropriate for the platform.")
)

social_media_node = functools.partial(agent_node, agent=social_media_agent, name="SOCIAL_MEDIA_AGENT")
print("Social Media Agent created successfully!")

Social Media Agent created successfully!


### Copy Editor Agent

The Copy Editor Agent refines social media posts to ensure they fit the target platform's tone and style.

In [22]:
copy_editor = create_agent(
    llm,
    [file_read, file_write, file_edit],
    ("You are an expert copy editor specializing in social media content. Your role is to "
     "review and refine social media posts to ensure they fit the tone and style of the "
     "specified social network. Read the social media post from 'social_media_post.txt' "
     "and edit it to make sure it's engaging, grammatically correct, and appropriate for "
     "the target platform and audience. Consider platform-specific best practices: "
     "LinkedIn (professional, informative), X (concise, engaging), Facebook (conversational, accessible). "
     "Save the final edited post to 'final_social_media_post.md' with markdown formatting.")
)

copy_editor_node = functools.partial(agent_node, agent=copy_editor, name="COPY_EDITOR")
print("Copy Editor Agent created successfully!")

Copy Editor Agent created successfully!


### Supervisor Agent

The Supervisor Agent coordinates the workflow between all agents.

In [23]:
supervisor = create_supervisor(
    llm,
    ("You are a supervisor coordinating a team to create social media posts from research papers. "
     "The team consists of: SEARCH_AGENT (finds papers), SOCIAL_MEDIA_AGENT (creates posts), "
     "and COPY_EDITOR (refines posts). Guide the workflow in this order: "
     "1. First, use SEARCH_AGENT to find and save the research paper content "
     "2. Then, use SOCIAL_MEDIA_AGENT to create a social media post from the paper "
     "3. Finally, use COPY_EDITOR to refine the post for the target platform "
     "When the final post is ready and saved, respond with FINISH."),
    ["SEARCH_AGENT", "SOCIAL_MEDIA_AGENT", "COPY_EDITOR"]
)

print("Supervisor Agent created successfully!")

Supervisor Agent created successfully!


/var/folders/11/l5r6prl962nbqt1wkfxvltm40000gn/T/ipykernel_5651/2020880168.py:50: LangChainDeprecationWarning: The method `BaseChatOpenAI.bind_functions` was deprecated in langchain-openai 0.2.1 and will be removed in 1.0.0. Use :meth:`~langchain_openai.chat_models.base.ChatOpenAI.bind_tools` instead.
  | llm.bind_functions(functions=[function_def], function_call="route")


## Graph Construction

Build the LangGraph workflow by connecting all agents and defining the flow logic.

In [24]:
# Create the state graph
workflow = StateGraph(ResearchPaperSocialMediaState)

# Add nodes for each agent
workflow.add_node("SEARCH_AGENT", search_node)
workflow.add_node("SOCIAL_MEDIA_AGENT", social_media_node)
workflow.add_node("COPY_EDITOR", copy_editor_node)
workflow.add_node("supervisor", supervisor)

# Add edges from agents back to supervisor
workflow.add_edge("SEARCH_AGENT", "supervisor")
workflow.add_edge("SOCIAL_MEDIA_AGENT", "supervisor")
workflow.add_edge("COPY_EDITOR", "supervisor")

# Add conditional edges from supervisor to agents
workflow.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "SEARCH_AGENT": "SEARCH_AGENT",
        "SOCIAL_MEDIA_AGENT": "SOCIAL_MEDIA_AGENT",
        "COPY_EDITOR": "COPY_EDITOR",
        "FINISH": END,
    },
)

# Set supervisor as entry point
workflow.set_entry_point("supervisor")

# Compile the graph
graph = workflow.compile()

print("LangGraph workflow compiled successfully!")

LangGraph workflow compiled successfully!


## System Execution Function

Create a function to execute the multi-agent system with the required parameters.

In [25]:
def create_social_media_post(paper_description: str, social_network: str, objective: str):
    """
    Create a social media post from a research paper.
    
    Args:
        paper_description: Description of the research paper to search for
        social_network: Target social network (LinkedIn, X, Facebook)
        objective: The objective/goal of the social media post
    """
    initial_message = (
        f"Create a social media post for {social_network} about the research paper: {paper_description}. "
        f"The objective is: {objective}. "
        f"First, search for and retrieve the paper content, then create an engaging post, "
        f"and finally edit it to fit the {social_network} platform style."
    )
    
    initial_state = {
        "messages": [HumanMessage(content=initial_message)],
        "paper_text": "",
        "social_network": social_network,
        "objective": objective,
        "social_media_post": "",
        "next": ""
    }
    
    print(f"🚀 Starting social media post creation for {social_network}...")
    print(f"📄 Paper: {paper_description}")
    print(f"🎯 Objective: {objective}")
    print("=" * 60)
    
    # Stream the workflow execution
    for step in graph.stream(initial_state, {"recursion_limit": 20}):
        if "__end__" not in step:
            for node_name, node_output in step.items():
                print(f"🤖 [{node_name}] Output:")
                if "messages" in node_output:
                    print(node_output["messages"][-1].content)
                elif "next" in node_output:
                    print(f"➡️ Next: {node_output['next']}")
                print("-" * 50)
    
    print("✅ Social media post creation completed!")
    
    # Check for output files and display final result
    output_files = list(WORKING_DIRECTORY.glob("*.md"))
    if not output_files:
        output_files = list(WORKING_DIRECTORY.glob("*.txt"))
    
    if output_files:
        print(f"📁 Output files created: {[f.name for f in output_files]}")
        
        # Find the final post file
        final_post_file = None
        for file in output_files:
            if "final" in file.name.lower() or file.name.endswith(".md"):
                final_post_file = file
                break
        
        if not final_post_file and output_files:
            final_post_file = max(output_files, key=lambda f: f.stat().st_mtime)
        
        if final_post_file:
            try:
                with open(final_post_file, 'r', encoding='utf-8') as f:
                    final_post = f.read()
                print(f"\n📱 Final social media post ({final_post_file.name}):")
                print("=" * 60)
                print(final_post)
                print("=" * 60)
            except Exception as e:
                print(f"❌ Error reading final post: {e}")
    else:
        print("⚠️ No output files were created. Check the execution logs above.")
    
    return graph

## Example Usage

Let's test our system with a sample research paper and create a LinkedIn post.

In [26]:
# Example parameters
paper_description = "QLoRA: Efficient Finetuning of Quantized LLMs"
social_network = "LinkedIn"
objective = "Explain the benefits of QLoRA to machine learning practitioners and researchers"

# Create the social media post
result = create_social_media_post(paper_description, social_network, objective)

🚀 Starting social media post creation for LinkedIn...
📄 Paper: QLoRA: Efficient Finetuning of Quantized LLMs
🎯 Objective: Explain the benefits of QLoRA to machine learning practitioners and researchers
🤖 [supervisor] Output:
➡️ Next: SEARCH_AGENT
--------------------------------------------------
🤖 [SEARCH_AGENT] Output:
Here's an engaging LinkedIn post about the research paper on QLoRA:

---

**Unlocking the Future of Machine Learning with QLoRA**

🚀 Exciting advancements in the field of machine learning are here! The recent paper titled **"QLoRA: Efficient Finetuning of Quantized LLMs"** introduces a groundbreaking approach to finetuning large language models (LLMs) with remarkable efficiency.

🔍 **What is QLoRA?**  
QLoRA stands for Quantized Low-Rank Adaptation, a method that allows for the finetuning of LLMs with up to 65 billion parameters using only 2/3/4-bit precision on consumer-grade GPUs. This innovation is a game-changer for researchers and practitioners who face memory con

## Alternative Example: Twitter/X Post

Let's create a different type of post for Twitter/X with a more concise approach.

In [27]:
# Example for Twitter/X
paper_description = "Attention Is All You Need - Transformer architecture"
social_network = "X"
objective = "Generate excitement about the revolutionary impact of Transformers in AI"

# Create the social media post
result = create_social_media_post(paper_description, social_network, objective)

🚀 Starting social media post creation for X...
📄 Paper: Attention Is All You Need - Transformer architecture
🎯 Objective: Generate excitement about the revolutionary impact of Transformers in AI
🤖 [supervisor] Output:
➡️ Next: SEARCH_AGENT
--------------------------------------------------
🤖 [SEARCH_AGENT] Output:
Here's the engaging social media post for X about the research paper on Transformer architecture:

---

🚀✨ Exciting times in AI! The revolutionary paper "Attention Is All You Need" introduced the Transformer architecture, changing the game for natural language processing and beyond! 🌍💡 

Transformers leverage attention mechanisms to process data in parallel, making them faster and more efficient than traditional models. This innovation has paved the way for groundbreaking applications in translation, content generation, and even healthcare! 🏥📚 

Join the AI revolution and explore how Transformers are reshaping our world! #AI #Transformers #MachineLearning #Innovation

--- 

T

## Custom Usage

You can modify the parameters below to test with different papers, social networks, and objectives.

In [28]:
# Customize these parameters for your own use case
custom_paper_description = "The Illusion of Thinking: Understanding the Strengths and Limitations of Reasoning Models via the Lens of Problem Complexity"  # Enter your paper description here
custom_social_network = "Facebook"     # Choose: LinkedIn, X, or Facebook
custom_objective = "Key message: while AI is powerful, it is still at the very heart a tool and it still does not have the ability to problem solve and generalize like someone trained in mathematical problem solving"          # Enter your objective here

# Uncomment and run to test with your custom parameters
# if custom_paper_description and custom_social_network and custom_objective:
#     result = create_social_media_post(custom_paper_description, custom_social_network, custom_objective)
# else:
#     print("Please fill in all custom parameters above to test your own use case.")

## System Summary

This multi-agent LangGraph system successfully demonstrates:

1. **Agent Coordination**: Multiple specialized agents working together through a supervisor
2. **Tool Integration**: File management and arXiv search capabilities
3. **Workflow Management**: Structured flow from research to final social media post
4. **Platform Adaptation**: Content tailored to different social media platforms
5. **LangSmith Integration**: Full tracing and monitoring capabilities

The system produces markdown-formatted social media posts that are optimized for the target platform and objective, demonstrating the power of multi-agent coordination in content creation workflows.